In [ ]:
import string
import os
import glob
import pandas as pd
import numpy as np

In [ ]:
# Download daily reports from ftp to temp folder

def get_cfs_data():
    wget_str = "wget -np --continue " \
    + "--user=$user " \
    + "--password=$passw " \
    + "--directory-prefix=$temp_dir " \
    + "ftp://ftp.sannet.gov/IN/SDPD/calls*"
    
    tmpl = string.Template(wget_str)
    command = tmpl.substitute(
        user='anonymous'
        passw='anonymous'
        temp_dir='/data/temp/pd_cfs'
    )
    os.system(command)

In [ ]:
get_cfs_data()

In [ ]:
# Data processing

def process_cfs_data():
    
    path = "/data/temp/pd_cfs"
    allFiles = glob.glob(os.path.join(path,"*.csv"))
    
    np_array_list = []
    
    for file_ in allFiles:
        df = pd.read_csv(file_, header=None, error_bad_lines=False)
        np_array_list.append(df.as_matrix())
        
    comb_np_array = np.vstack(np_array_list)
    
    temp_frame = pd.DataFrame(comb_np_array)
    curr_frame = pd.read_csv('/data/prod/pd_calls_for_service_2016_datasd.csv')
    
    columns_names = current.columns.values
    temp_frame.columns = columns_names
    
    prod_frame = curr_frame.append(temp_frame, ignore_index=True)
    prod_frame = prod_frame.drop_duplicates(subset=['incident_num'])
    
    prod_frame['date_time'] = pd.to_datetime(prod_frame['date_time'])
    prod_frame['day'] = prod_frame['day'].astype(int)
    prod_frame['stno'] = prod_frame['stno'].astype(int)
    prod_frame['beat'] = pd.to_numeric(prod_frame['beat'], errors='coerce')
    prod_frame['priority'] = pd.to_numeric(prod_frame['priority'],errors='coerce')
    
    prod_frame.to_csv('/data/prod/pd_calls_for_service_2016_datasd.csv', index=False, encoding='utf-8')

In [ ]:
process_cfs_data()